# Pima Indians Diabetes Database

[About Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)
## Context

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

## Content
The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

## Acknowledgements
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

Inspiration
Can you build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?

In [1]:
import random
import numpy as np

random.seed(42)
np.random.seed(100)

In [2]:
import pandas as pd
import os
from f_importance.util.runner import compute_importance

In [3]:
diabetes = pd.read_csv("./dataset/diabetes.csv")

diabetes.sample(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
173,1,79,60,42,48,43.5,0.678,23,0
253,0,86,68,32,0,35.8,0.238,25,0
207,5,162,104,0,0,37.7,0.151,52,1
737,8,65,72,23,0,32.0,0.600,42,0
191,9,123,70,44,94,33.1,0.374,40,0
754,8,154,78,32,0,32.4,0.443,45,1
159,17,163,72,41,114,40.9,0.817,47,1
448,0,104,64,37,64,33.6,0.510,22,1
359,1,196,76,36,249,36.5,0.875,29,1
651,1,117,60,23,106,33.8,0.466,27,0


In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [5]:
def train_evaluate(data: pd.DataFrame, clazz= XGBClassifier, target="Outcome", proba=False, metrics=metrics.accuracy_score, n_fold=5, **kwargs):
    X = data.drop(columns=[target])
    y = data[target]
    score = 0
    for train_index, test_index in StratifiedKFold(
                n_fold, shuffle=True, random_state=42
            ).split(X, y):
        (X_train, y_train), (X_test, y_test) = (
            (X.loc[train_index], y.loc[train_index]),
            (X.loc[test_index], y.loc[test_index]),
        )
        model: XGBClassifier = clazz(**kwargs)
        model.fit(X_train, y_train)
        if proba:
            y_preds = model.predict_proba(X_test)[:, 1]
        else:
            y_preds = model.predict(X_test)
        score += metrics(y_test, y_preds)
    return score / n_fold

In [6]:
print("Accuracy Score : ", train_evaluate(diabetes))

Accuracy Score :  0.7434173669467787


In [7]:
importance = compute_importance(
    model_name = "XGBClassifier",
    method = "DataFold",
    metric = "accuracy_score",
    dataset  = diabetes,
    targets = "Outcome",
    n_gram = (1, 1),
    val_rate = 0.15,
    shuffle = True,
    n = 5,
    is_regression = False,
    n_jobs = os.cpu_count(),
    refit = True,
)

In [8]:
importance

,Importance,Split0,Split1,Split2,Split3,Split4
,0.739504,0.792208,0.740260,0.746753,0.725490,0.692810
'Glucose',0.062372,0.623377,0.688312,0.681818,0.686275,0.705882
'BMI',0.025966,0.759740,0.727273,0.655844,0.732026,0.692810
'BloodPressure',0.011629,0.707792,0.681818,0.785714,0.784314,0.679739
'DiabetesPedigreeFunction',0.009040,0.766234,0.701299,0.727273,0.725490,0.732026
'Pregnancies',0.007639,0.642857,0.798701,0.681818,0.816993,0.718954
'Age',0.006400,0.694805,0.727273,0.753247,0.725490,0.764706
'Insulin',-0.005331,0.688312,0.746753,0.766234,0.738562,0.784314
'SkinThickness',-0.009227,0.759740,0.785714,0.675325,0.745098,0.777778


In [9]:
to_delete = ['SkinThickness', 'Insulin']


to_delete

['SkinThickness', 'Insulin']

In [10]:
print("Accuracy Score : ", train_evaluate(diabetes.drop(columns=[to_delete[0]])))

Accuracy Score :  0.7370596723537901


In [11]:
print("Accuracy Score : ", train_evaluate(diabetes.drop(columns=[to_delete[1]])))

Accuracy Score :  0.747415329768271


In [12]:
print("Accuracy Score : ", train_evaluate(diabetes.drop(columns=['Age'])))

Accuracy Score :  0.7188269247092777


In [13]:
importance2 = compute_importance(
    model_name = "XGBClassifier",
    method = "DataFold",
    metric = "accuracy_score",
    dataset  = diabetes,
    targets = "Outcome",
    n_gram = (1, 2),
    val_rate = 0.15,
    shuffle = True,
    n = 5,
    is_regression = False,
    n_jobs = os.cpu_count(),
    refit = True
)

In [14]:
importance2

,Importance,Split0,Split1,Split2,Split3,Split4
,0.722698,0.714286,0.701299,0.720779,0.738562,0.738562
"'Glucose', 'Age'",0.077014,0.701299,0.649351,0.701299,0.588235,0.588235
"'Glucose', 'Insulin'",0.062643,0.714286,0.655844,0.688312,0.607843,0.633987
'Glucose',0.054741,0.675325,0.701299,0.636364,0.647059,0.679739
"'Pregnancies', 'Glucose'",0.052067,0.694805,0.655844,0.616883,0.699346,0.686275
"'Glucose', 'DiabetesPedigreeFunction'",0.046923,0.668831,0.681818,0.681818,0.686275,0.660131
"'Glucose', 'BMI'",0.046906,0.662338,0.675325,0.681818,0.666667,0.692810
"'Glucose', 'SkinThickness'",0.035124,0.655844,0.636364,0.714286,0.718954,0.712418
"'Glucose', 'BloodPressure'",0.027375,0.642857,0.714286,0.720779,0.718954,0.679739
"'BMI', 'DiabetesPedigreeFunction'",0.013004,0.668831,0.655844,0.759740,0.758170,0.705882
